In [1]:
import numpy as np
import quantum as q
import matplotlib.pyplot as plt
from math import sqrt
from tqdm import trange

In [2]:
number_of_states = 10**4
number_of_projections = 10**4
bins = 1000
d = 2

In [3]:
p = []
neg = []

count = 0
psi = np.zeros(d**2, dtype = complex)
psi[np.random.randint(d**2) - 1] = 1
rho = np.outer(psi, psi.conj())

`V` determines the initial state, whereas `U` determines the projection basis

`test_state` samples a random state from Haar measure by picking a random unitary `V`. The random state $|\phi\rangle$ is given by $|\phi\rangle = V |\psi\rangle$.

`test_proj` projects the given state $|\phi\rangle$ onto a random basis defined by `U` and returns the negativity of the post measurement state.

`test_state` then return samples from $P(N_f | \phi)$.

In [4]:
def test_state(i):
    V = q.randu(d**2)
    def test_proj(j):
        U = q.randu(d**2)
        rho_post = (
            U.conj().transpose()
            @np.diag((U@V@rho@V.conj().transpose()@U.conj().transpose()).diagonal())
            @U
        )
        
        return q.negativity(rho_post, [d, d], [0, 1])
    
    return [test_proj(j) for j in range(number_of_projections)]

In [5]:
def statistical_distance(p, q):
    if (len(p) != len(q)):
        print('p and q are not of the same length')
        return None
    return 0.5*sum([abs(p[i] - q[i]) for i in range(len(p))])

In [6]:
neg = [test_state(i) for i in trange(number_of_states)]

100%|██████████| 10000/10000 [6:02:16<00:00,  2.01s/it]  


In [7]:
mean_distribution, _ = np.histogram(np.concatenate(neg), bins = bins, range = (-1e-3, (d-1)/2))
mean_distribution = mean_distribution/(number_of_states*number_of_projections)

In [8]:
distances = []
ps = []

for neg_i in neg:
    p = np.histogram(neg_i, bins = bins, range = (-1e-3, (d-1)/2))[0]
    p = p/number_of_projections
    ps.append(p)
    distances.append(statistical_distance(p, mean_distribution))
    
print('d', d)
print('number of states', number_of_states)
print('number of projections', number_of_projections)
print('bins', bins)
print('average distance {}'.format(sum(distances)/len(distances)))

d 2
number of states 10000
number of projections 10000
bins 1000
average distance 0.06109950237299993
